# 房價預測(Kaggle)
以下用房價預測資料, 觀察均值編碼的效果

In [2]:
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

data_path = 'data/'
df_train = pd.read_csv(data_path + 'house_train.csv')
df_test = pd.read_csv(data_path + 'house_test.csv')

train_Y = np.log1p(df_train['SalePrice'])
ids = df_test['Id']
df_train = df_train.drop(['Id', 'SalePrice'], axis =1)
df_test = df_test.drop(['Id'], axis = 1)
df = pd.concat([df_train, df_test])
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [7]:
# 找出object欄位
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f"{len(object_features)} object Features : {object_features}\n")
# 只保留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

43 object Features : ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']



,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
1,RL,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
2,RL,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal
3,RL,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,None,None,None,WD,Abnorml
4,RL,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,None,None,None,WD,Normal


In [11]:
# 對照組 : 標籤編碼 + 線性迴歸
df_le = pd.DataFrame()
for i in df.columns:
    df_le[i] = LabelEncoder().fit_transform(df[i])
train_X = df_le[:train_num]
estimator = LinearRegression()
start = time.time()
print(f"shape: {train_X.shape}")
print(f"score: {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}")
print(f"time: {time.time()-start} sec")    

shape: (1460, 43)
score: 0.6615606866851302
time: 0.018616676330566406 sec


In [12]:
# enc + linear
df_enc = df_le
df_enc = OneHotEncoder().fit_transform(df_enc).toarray()
df_enc = pd.DataFrame(df_enc)
train_X = df_enc[:train_num]
estimator = LinearRegression()
start = time.time()
print(f"shape: {train_X.shape}")
print(f"score: {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}")
print(f"time: {time.time()-start} sec")  

shape: (1460, 274)
score: -7.147129671268858e+22
time: 0.15871882438659668 sec


In [23]:
# 均值編碼 + 線性迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
for i in df.columns:
    mean_df = data.groupby([i])['SalePrice'].mean().reset_index()
    mean_df.columns = [i, f'{i}_mean']
    data = pd.merge(data, mean_df, on=i, how='left')
    data = data.drop([i], axis =1)
data = data.drop(['SalePrice'] , axis=1)
estimator = LinearRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (1460, 274)
score : 0.7624230403716973
time : 0.015841960906982422 sec


In [24]:
# 對照組 : 標籤編碼 + gradient
train_X = df_le[:train_num]
estimator = GradientBoostingRegressor()
start = time.time()
print(f"shape: {train_X.shape}")
print(f"score: {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}")
print(f"time: {time.time()-start} sec")  

shape: (1460, 43)
score: 0.7772640391993063
time: 0.874445915222168 sec


In [25]:
# enc + gradient
train_X = df_enc[:train_num]
estimator = GradientBoostingRegressor()
start = time.time()
print(f"shape: {train_X.shape}")
print(f"score: {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}")
print(f"time: {time.time()-start} sec")  

shape: (1460, 274)
score: 0.7903462653730512
time: 3.2978997230529785 sec


In [29]:
# 均值編碼 + 線性迴歸
estimator = GradientBoostingRegressor()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (1460, 43)
score : 0.8059091225553315
time : 0.7221736907958984 sec


# [鐵達尼生存預測(Kaggle)](https://www.kaggle.com/c/titanic)
# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [31]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

In [33]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_le = pd.DataFrame()
for i in df.columns:
    df_le[i] = LabelEncoder().fit_transform(df[i])
train_X = df_le[:train_num]
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.780004837244799
time : 0.031249523162841797 sec


In [35]:
# OneHotencoding+ 邏輯斯迴歸
df_enc = df_le
df_enc = OneHotEncoder().fit_transform(df_enc).toarray()
df_enc = pd.DataFrame(df_enc)
train_X = df_enc[:train_num]
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 2429)
score : 0.8013346043513216
time : 0.10515022277832031 sec


In [41]:
# 均值編碼 + 邏輯回歸
data = pd.concat([df[:train_num], train_Y], axis = 1)
for i in df.columns:
    mean_df = data.groupby([i])['Survived'].mean().reset_index()
    mean_df.columns = [i, f'{i}_mean']
    data = pd.merge(data, mean_df, on=i, how='left')
    data = data.drop([i], axis=1)
data = data.drop(['Survived'], axis=1)
estimator = LogisticRegression()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 1.0
time : 0.01636505126953125 sec


均值編碼在訓練資料中的表現高達1，應以造成Overfitting

In [44]:
# OneHotencoding + gradient
train_X = df_le[:train_num]
estimator = GradientBoostingClassifier()
start = time.time()
print(f"shape: {train_X.shape}")
print(f"score: {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}")
print(f"time: {time.time()-start} sec")  

shape: (891, 5)
score: 0.7991313528291322
time: 0.3432469367980957 sec


In [45]:
# OneHotencoding + mean
estimator = GradientBoostingRegressor()
start = time.time()
print(f'shape : {data.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.9999996032710972
time : 0.12995123863220215 sec
